<a href="https://colab.research.google.com/github/WinNatch/Modelling-Individual-Assessment/blob/main/Q2_Individual_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Intall and Import Package/Library
!pip install pulp
from pulp import *

##Question A

In [12]:
model = LpProblem("Q2", sense=LpMinimize)

Facility = ['Cincinnati','Oakland']
Railroad = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

Demand = {1:22418, 2:6800, 3:80290, 4:100447, 5:24570, 6:64761, 7:33689, 8:68486, 9:148586, 10:111475, 11:112000}
Cincinnati_T_Cost = {1:10000, 2:0.84, 3:0.83, 4:0.45, 5:0.36, 6:10000, 7:10000, 8:0.34, 9:0.34, 10:0.34, 11:10000}
Oakland_T_Cost = {1:0.22, 2:0.74, 3:0.49, 4:10000, 5:10000, 6:0.22, 7:0.22, 8:10000, 9:10000, 10:10000, 11:0.15}
Prod_Cost = {'Cincinnati':1.2, 'Oakland':1.65}

# Desicion Variables
Qt = LpVariable.dicts("Q", [(i,j) for i in Facility for j in Railroad], lowBound=0, cat='Continuous')

# Objective Function
model += lpSum(Qt['Cincinnati',j] * (Prod_Cost['Cincinnati'] + Cincinnati_T_Cost[j]) for j in Railroad) \
         + lpSum(Qt['Oakland',j] * (Prod_Cost['Oakland'] + Oakland_T_Cost[j]) for j in Railroad)

# Constraints
for j in Railroad:
    model += lpSum(Qt[i,j] for i in Facility) == Demand[j]

for i in Facility:
    model += lpSum(Qt[i,j] for j in Railroad) <= 500000

# Solve the Problem
model.writeLP("Q2.lp")
model.solve()

# The Status of the Solution
print("Model Status:{}".format(LpStatus[model.status]))

# To Display Optimal Decision Variables & Reduced Cost per Variable
for v in model.variables():
	  print(v.name, "=", v.varValue)

# To Display Optimal Value of Objective Function
print("Objective=", value(model.objective))

Model Status:Optimal
Q_('Cincinnati',_1) = 0.0
Q_('Cincinnati',_10) = 111475.0
Q_('Cincinnati',_11) = 0.0
Q_('Cincinnati',_2) = 6800.0
Q_('Cincinnati',_3) = 39636.0
Q_('Cincinnati',_4) = 100447.0
Q_('Cincinnati',_5) = 24570.0
Q_('Cincinnati',_6) = 0.0
Q_('Cincinnati',_7) = 0.0
Q_('Cincinnati',_8) = 68486.0
Q_('Cincinnati',_9) = 148586.0
Q_('Oakland',_1) = 22418.0
Q_('Oakland',_10) = 0.0
Q_('Oakland',_11) = 112000.0
Q_('Oakland',_2) = 0.0
Q_('Oakland',_3) = 40654.0
Q_('Oakland',_4) = 0.0
Q_('Oakland',_5) = 0.0
Q_('Oakland',_6) = 64761.0
Q_('Oakland',_7) = 33689.0
Q_('Oakland',_8) = 0.0
Q_('Oakland',_9) = 0.0
Objective= 1318984.93


In [23]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value

model = LpProblem("Q2", sense=LpMinimize)

Facility = ['Cincinnati', 'Oakland']
Railroad = {1: 'Santa Ana', 2: 'El Paso', 3: 'Pendleton', 4: 'Houston', 5: 'Kansas City', 6: 'Los Angeles',
            7: 'Glendale', 8: 'Jacksonville', 9: 'Little Rock', 10: 'Bridgeport', 11: 'Sacramento'}

Demand = {1: 22418, 2: 6800, 3: 80290, 4: 100447, 5: 24570, 6: 64761, 7: 33689, 8: 68486, 9: 148586, 10: 111475, 11: 112000}
Cincinnati_T_Cost = {1: 10000, 2: 0.84, 3: 0.83, 4: 0.45, 5: 0.36, 6: 10000, 7: 10000, 8: 0.34, 9: 0.34, 10: 0.34, 11: 10000}
Oakland_T_Cost = {1: 0.22, 2: 0.74, 3: 0.49, 4: 10000, 5: 10000, 6: 0.22, 7: 0.22, 8: 10000, 9: 10000, 10: 10000, 11: 0.15}
Prod_Cost = {'Cincinnati': 1.2, 'Oakland': 1.65}

# Decision Variables
Qt = LpVariable.dicts("Q", [(i, j) for i in Facility for j in Railroad.keys()], lowBound=0, cat='Continuous')

# Objective Function
model += lpSum(Qt['Cincinnati', j] * (Prod_Cost['Cincinnati'] + Cincinnati_T_Cost[j]) for j in Railroad.keys()) + \
         lpSum(Qt['Oakland', j] * (Prod_Cost['Oakland'] + Oakland_T_Cost[j]) for j in Railroad.keys())

# Constraints
for j in Railroad.keys():
    model += lpSum(Qt[i, j] for i in Facility) == Demand[j]

model += lpSum(Qt['Cincinnati', j] for j in Railroad.keys()) <= 500000

# Solve the Problem
model.writeLP("Q2.lp")
model.solve()

# The Status of the Solution
print("Model Status: {}".format(LpStatus[model.status]))

# Function to replace numbers with Railroad names in variable names
def replace_with_railroad_name(var_name):
    parts = var_name.split('_')
    if parts[-1].strip('()').isdigit():
        parts[-1] = Railroad[int(parts[-1].strip('()'))] + parts[-1][-1]
    return '_'.join(parts)

# Prepare lists for Cincinnati and Oakland
cincinnati_vars = []
oakland_vars = []
bridgeport_sacramento_cincinnati = []
bridgeport_sacramento_oakland = []

# Separate variables for Cincinnati and Oakland, moving Bridgeport and Sacramento to the bottom
for v in model.variables():
    var_name_converted = replace_with_railroad_name(v.name)
    var_value = v.varValue
    if "Cincinnati" in var_name_converted:
        if "Bridgeport" in var_name_converted or "Sacramento" in var_name_converted:
            bridgeport_sacramento_cincinnati.append((var_name_converted, var_value))
        else:
            cincinnati_vars.append((var_name_converted, var_value))
    elif "Oakland" in var_name_converted:
        if "Bridgeport" in var_name_converted or "Sacramento" in var_name_converted:
            bridgeport_sacramento_oakland.append((var_name_converted, var_value))
        else:
            oakland_vars.append((var_name_converted, var_value))

# Append Bridgeport and Sacramento at the end
cincinnati_vars.extend(bridgeport_sacramento_cincinnati)
oakland_vars.extend(bridgeport_sacramento_oakland)

# Determine the maximum length for formatting
max_len = max(len(cincinnati_vars), len(oakland_vars))

# Pad lists to make them equal length
while len(cincinnati_vars) < max_len:
    cincinnati_vars.append(("", ""))
while len(oakland_vars) < max_len:
    oakland_vars.append(("", ""))

# Print the results in separate columns for Cincinnati and Oakland
print("\nOptimal Shipment Plan")
print(f"{'Cincinnati':<35} {'Value':<14} {'Oakland':<31} {'Value'}")
print(f"{'-'*35} {'-'*14} {'-'*31} {'-'*14}")

for (cinc_name, cinc_value), (oak_name, oak_value) in zip(cincinnati_vars, oakland_vars):
    print(f"{cinc_name:<35} {cinc_value:<14} {oak_name:<31} {oak_value}")

# Calculate total allocations for Cincinnati and Oakland
total_cincinnati = sum(Qt['Cincinnati', j].varValue for j in Railroad)
total_oakland = sum(Qt['Oakland', j].varValue for j in Railroad)

# Print the results with the calculated values
print(f"\n{'Total Cincinnati Allocation:':<35} {total_cincinnati:<14} {'Total Oakland Allocation:':<31} {total_oakland:<14}")

# Display Optimal Value of Objective Function
print("\nTotal Cost =", value(model.objective))

Model Status: Optimal

Optimal Shipment Plan
Cincinnati                          Value          Oakland                         Value
----------------------------------- -------------- ------------------------------- --------------
Q_('Cincinnati',_Santa Ana)         0.0            Q_('Oakland',_Santa Ana)        22418.0
Q_('Cincinnati',_El Paso)           6800.0         Q_('Oakland',_El Paso)          0.0
Q_('Cincinnati',_Pendleton)         39636.0        Q_('Oakland',_Pendleton)        40654.0
Q_('Cincinnati',_Houston)           100447.0       Q_('Oakland',_Houston)          0.0
Q_('Cincinnati',_Kansas City)       24570.0        Q_('Oakland',_Kansas City)      0.0
Q_('Cincinnati',_Los Angeles)       0.0            Q_('Oakland',_Los Angeles)      64761.0
Q_('Cincinnati',_Glendale)          0.0            Q_('Oakland',_Glendale)         33689.0
Q_('Cincinnati',_Jacksonville)      68486.0        Q_('Oakland',_Jacksonville)     0.0
Q_('Cincinnati',_Little Rock)       148586.0       Q_('O

##Question B

In [ ]:
total_sale = sum(Qt[i,j].varValue for i in Facility for j in Railroad)
total_sale

773522.0

In [ ]:
avg_cost = round(value(model.objective)/sum(Qt[i,j].varValue for i in Facility for j in Railroad),2)
avg_cost

1.71

##Question C

In [ ]:
selling_price = round(avg_cost * 1.15,2)
selling_price

1.97

In [ ]:
total_profit = round((selling_price * total_sale) - value(model.objective), 2)
total_profit

204853.41

##Question D

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpStatus

def solve_model(Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost):
    # Define Environment & Direction of Optimization
    model = LpProblem("Q2", sense=LpMinimize)

    Facility = ['Cincinnati', 'Oakland']
    Railroad = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

    Demand = {1: 22418, 2: 6800, 3: 80290, 4: 100447, 5: 24570, 6: 64761, 7: 33689, 8: 68486, 9: 148586, 10: 111475, 11: 112000}

    # Decision Variables
    Qt = LpVariable.dicts("Quantity", [(i, j) for i in Facility for j in Railroad], lowBound=0, cat='Continuous')

    # Objective Function
    model += lpSum(Qt['Cincinnati', j] * (Prod_Cost['Cincinnati'] + Cincinnati_T_Cost[j]) for j in Railroad) + \
             lpSum(Qt['Oakland', j] * (Prod_Cost['Oakland'] + Oakland_T_Cost[j]) for j in Railroad)

    # Constraints
    for j in Railroad:
        model += lpSum(Qt[i, j] for i in Facility) == Demand[j]

    model += lpSum(Qt['Cincinnati', j] for j in Railroad) <= 500000

    # Solve the Problem
    model.solve()

    return value(model.objective), {v.name: v.varValue for v in model.variables()}

# Base Costs
Cincinnati_T_Cost = {1: 10000, 2: 0.84, 3: 0.83, 4: 0.45, 5: 0.36, 6: 10000, 7: 10000, 8: 0.34, 9: 0.34, 10: 0.34, 11: 10000}
Oakland_T_Cost = {1: 0.22, 2: 0.74, 3: 0.49, 4: 10000, 5: 10000, 6: 0.22, 7: 0.22, 8: 10000, 9: 10000, 10: 10000, 11: 0.15}
Prod_Cost = {'Cincinnati': 1.2, 'Oakland': 1.65}

# Increase costs by 10%
Cincinnati_T_Cost_increase_10 = {k: v*1.1 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_increase_10 = {k: v*1.1 for k, v in Oakland_T_Cost.items()}
Cincinnati_T_Cost_increase_20 = {k: v*1.2 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_increase_20 = {k: v*1.2 for k, v in Oakland_T_Cost.items()}
Cincinnati_T_Cost_increase_30 = {k: v*1.3 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_increase_30 = {k: v*1.3 for k, v in Oakland_T_Cost.items()}

# Decrease costs by 10%
Cincinnati_T_Cost_decrease_10 = {k: v*0.9 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_decrease_10 = {k: v*0.9 for k, v in Oakland_T_Cost.items()}
Cincinnati_T_Cost_decrease_20 = {k: v*0.8 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_decrease_20 = {k: v*0.8 for k, v in Oakland_T_Cost.items()}
Cincinnati_T_Cost_decrease_30 = {k: v*0.7 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_decrease_30 = {k: v*0.7 for k, v in Oakland_T_Cost.items()}

# Run models with different fuel price scenarios
T_Cost_original, _ = solve_model(Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost)
T_Cost_increase_10, _ = solve_model(Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost)
T_Cost_decrease_10, _ = solve_model(Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost)
T_Cost_increase_20, _ = solve_model(Cincinnati_T_Cost_increase_20, Oakland_T_Cost_increase_20, Prod_Cost)
T_Cost_decrease_20, _ = solve_model(Cincinnati_T_Cost_decrease_20, Oakland_T_Cost_decrease_20, Prod_Cost)
T_Cost_increase_30, _ = solve_model(Cincinnati_T_Cost_increase_30, Oakland_T_Cost_increase_30, Prod_Cost)
T_Cost_decrease_30, _ = solve_model(Cincinnati_T_Cost_decrease_30, Oakland_T_Cost_decrease_30, Prod_Cost)

print(f"Origianal Total Cost: {T_Cost_original}")
print(f"Total Cost with 10% Increase in Freight Costs: {round(T_Cost_increase_10, 2)}")
print(f"Total Cost with 20% Increase in Freight Costs: {round(T_Cost_increase_20, 2)}")
print(f"Total Cost with 30% Increase in Freight Costs: {round(T_Cost_increase_30, 2)}")

print(f"\n5% Mark-up Bid Price: {round((1.05*T_Cost_original)/total_sale, 2)}")
print(f"10% Mark-up Bid Price: {round((1.10*T_Cost_original)/total_sale, 2)}")
print(f"15% Mark-up Bid Price: {round((1.15*T_Cost_original)/total_sale, 2)}")

print(f"\nCurrent Breakeven Bid Price: {round(T_Cost_original/total_sale, 2)}")
print(f"Breakeven Bid Price with 10% Increase in Freight Costs: {round(T_Cost_increase_10/total_sale, 2)}")
print(f"Breakeven Bid Price with 20% Increase in Freight Costs: {round(T_Cost_increase_20/total_sale, 2)}")
print(f"Breakeven Bid Price with 30% Increase in Freight Costs: {round(T_Cost_increase_30/total_sale, 2)}")

Origianal Total Cost: 1318984.93
Total Cost with 10% Increase in Freight Costs: 1345752.29
Total Cost with 20% Increase in Freight Costs: 1372519.66
Total Cost with 30% Increase in Freight Costs: 1399287.02

5% Mark-up Bid Price: 1.79
10% Mark-up Bid Price: 1.88
15% Mark-up Bid Price: 1.96

Current Breakeven Bid Price: 1.71
Breakeven Bid Price with 10% Increase in Freight Costs: 1.74
Breakeven Bid Price with 20% Increase in Freight Costs: 1.77
Breakeven Bid Price with 30% Increase in Freight Costs: 1.81


##Question E

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpStatus

def solve_model(Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost):
    # Define Environment & Direction of Optimization
    model = LpProblem("Q2", sense=LpMinimize)

    Facility = ['Cincinnati', 'Oakland']
    Railroad = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

    Demand = {1: 22418, 2: 6800, 3: 80290, 4: 100447, 5: 24570, 6: 64761, 7: 33689, 8: 68486, 9: 148586, 10: 111475, 11: 112000}

    # Decision Variables
    Qt = LpVariable.dicts("Quantity", [(i, j) for i in Facility for j in Railroad], lowBound=0, cat='Continuous')

    # Objective Function
    model += lpSum(Qt['Cincinnati', j] * (Prod_Cost['Cincinnati'] + Cincinnati_T_Cost[j]) for j in Railroad) + \
             lpSum(Qt['Oakland', j] * (Prod_Cost['Oakland'] + Oakland_T_Cost[j]) for j in Railroad)

    # Constraints
    for j in Railroad:
        model += lpSum(Qt[i, j] for i in Facility) == Demand[j]

    model += lpSum(Qt['Cincinnati', j] for j in Railroad) <= 500000

    # Solve the Problem
    model.solve()

    return round(value(model.objective), 2), {v.name: round(v.varValue, 2) for v in model.variables()}

# Base Costs
Cincinnati_T_Cost = {1: 10000, 2: 0.84, 3: 0.83, 4: 0.45, 5: 0.36, 6: 10000, 7: 10000, 8: 0.34, 9: 0.34, 10: 0.34, 11: 10000}
Oakland_T_Cost = {1: 0.22, 2: 0.74, 3: 0.49, 4: 10000, 5: 10000, 6: 0.22, 7: 0.22, 8: 10000, 9: 10000, 10: 10000, 11: 0.15}
Prod_Cost = {'Cincinnati': 1.2, 'Oakland': 1.65}

# Increase costs by 10%
Cincinnati_T_Cost_increase_10 = {k: v*1.1 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_increase_10 = {k: v*1.1 for k, v in Oakland_T_Cost.items()}
Prod_Cost_increase_10 = {k: v*1.1 for k, v in Prod_Cost.items()}

# Decrease costs by 10%
Cincinnati_T_Cost_decrease_10 = {k: v*0.9 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_decrease_10 = {k: v*0.9 for k, v in Oakland_T_Cost.items()}
Prod_Cost_decrease_10 = {k: v*0.9 for k, v in Prod_Cost.items()}

# Total sales (sum of all demand)
total_sale = sum(Demand.values())

# Run models for all 9 scenarios
scenarios = {
    'Original': (Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost),
    'Increase Both 10%': (Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost_increase_10),
    'Decrease Both 10%': (Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost_decrease_10),
    'Increase Transportation 10%': (Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost),
    'Decrease Transportation 10%': (Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost),
    'Increase Production 10%': (Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost_increase_10),
    'Decrease Production 10%': (Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost_decrease_10),
    'Increase Trans Decrease Prod 10%': (Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost_decrease_10),
    'Decrease Trans Increase Prod 10%': (Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost_increase_10)
}

# Solve and print results for each scenario
print(f"{'Scenario':<45} {'Objective':<20} {'Average Cost per Gallon':<20}")
for scenario, (cin_cost, oak_cost, prod_cost) in scenarios.items():
    objective, _ = solve_model(cin_cost, oak_cost, prod_cost)
    avg_cost_per_gallon = round(objective / total_sale, 2)
    print(f"{scenario:<45} {objective:<20} {avg_cost_per_gallon:<20.2f}")

Scenario                                      Objective            Average Cost per Gallon
Original                                      1318984.93           1.71                
Increase Both 10%                             1450883.42           1.88                
Decrease Both 10%                             1187086.44           1.53                
Increase Transportation 10%                   1345752.29           1.74                
Decrease Transportation 10%                   1292217.57           1.67                
Increase Production 10%                       1424116.06           1.84                
Decrease Production 10%                       1213853.8            1.57                
Increase Trans Decrease Prod 10%              1240621.16           1.60                
Decrease Trans Increase Prod 10%              1397348.7            1.81                


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpStatus

def solve_model(Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost):
    # Define Environment & Direction of Optimization
    model = LpProblem("Q2", sense=LpMinimize)

    Facility = ['Cincinnati', 'Oakland']
    Railroad = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

    Demand = {1: 22418, 2: 6800, 3: 80290, 4: 100447, 5: 24570, 6: 64761, 7: 33689, 8: 68486, 9: 148586, 10: 111475, 11: 112000}

    # Decision Variables
    Qt = LpVariable.dicts("Quantity", [(i, j) for i in Facility for j in Railroad], lowBound=0, cat='Continuous')

    # Objective Function
    model += lpSum(Qt['Cincinnati', j] * (Prod_Cost['Cincinnati'] + Cincinnati_T_Cost[j]) for j in Railroad) + \
             lpSum(Qt['Oakland', j] * (Prod_Cost['Oakland'] + Oakland_T_Cost[j]) for j in Railroad)

    # Constraints
    for j in Railroad:
        model += lpSum(Qt[i, j] for i in Facility) == Demand[j]

    model += lpSum(Qt['Cincinnati', j] for j in Railroad) <= 500000

    # Solve the Problem
    model.solve()

    return round(value(model.objective), 2), {v.name: round(v.varValue, 2) for v in model.variables()}

# Base Costs
Cincinnati_T_Cost = {1: 10000, 2: 0.84, 3: 0.83, 4: 0.45, 5: 0.36, 6: 10000, 7: 10000, 8: 0.34, 9: 0.34, 10: 0.34, 11: 10000}
Oakland_T_Cost = {1: 0.22, 2: 0.74, 3: 0.49, 4: 10000, 5: 10000, 6: 0.22, 7: 0.22, 8: 10000, 9: 10000, 10: 10000, 11: 0.15}
Prod_Cost = {'Cincinnati': 1.2, 'Oakland': 1.65}

# Increase costs by 10%
Cincinnati_T_Cost_increase_10 = {k: v*1.1 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_increase_10 = {k: v*1.1 for k, v in Oakland_T_Cost.items()}
Prod_Cost_increase_10 = {k: v*1.1 for k, v in Prod_Cost.items()}

# Decrease costs by 10%
Cincinnati_T_Cost_decrease_10 = {k: v*0.9 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_decrease_10 = {k: v*0.9 for k, v in Oakland_T_Cost.items()}
Prod_Cost_decrease_10 = {k: v*0.9 for k, v in Prod_Cost.items()}

# Total sales (sum of all demand)
total_sale = sum(Demand.values())

# Run models for all 9 scenarios
scenarios = {
    'Original': (Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost),
    'Increase Both 10%': (Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost_increase_10),
    'Decrease Both 10%': (Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost_decrease_10),
    'Increase Transportation 10%': (Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost),
    'Decrease Transportation 10%': (Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost),
    'Increase Production 10%': (Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost_increase_10),
    'Decrease Production 10%': (Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost_decrease_10),
    'Increase Trans Decrease Prod 10%': (Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost_decrease_10),
    'Decrease Trans Increase Prod 10%': (Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost_increase_10)
}

# Solve and store results for each scenario
results = {}
for scenario, (cin_cost, oak_cost, prod_cost) in scenarios.items():
    results[scenario] = solve_model(cin_cost, oak_cost, prod_cost)

# Print results with changes compared to the original in a more presentable format
print(f"{'Scenario':<20} {'Objective':<20} {'Avg Cost per Gallon':<15}")
for scenario, (objective, variables) in results.items():
    avg_cost_per_gallon = round(objective / total_sale, 2)
    print(f"\n{scenario:<25} {objective:<21} {avg_cost_per_gallon:<20.2f}")

    # Create columns for quantities of Cincinnati and Oakland
    cincinnati_quantities = {j: 0 for j in range(1, 12)}
    oakland_quantities = {j: 0 for j in range(1, 12)}

    for var, value in variables.items():
        if "Cincinnati" in var:
            index = var.split('_')[-1].replace(')', '')
            cincinnati_quantities[int(index)] = value
        elif "Oakland" in var:
            index = var.split('_')[-1].replace(')', '')
            oakland_quantities[int(index)] = value

    # Print table for each scenario
    print("\n| Railroad | Cincinnati Quantity | Oakland Quantity |")
    print("|----------|---------------------|------------------|")
    for j in range(1, 12):
        print(f"| {j:<8} | {cincinnati_quantities[j]:<19} | {oakland_quantities[j]:<16} |")

    # Compare variables with original scenario
    if scenario != 'Original':
        print("\nChanges in variable values:")
        original_variables = results['Original'][1]
        for j in range(1, 12):
            change_cincinnati = cincinnati_quantities[j] - original_variables.get(f"Quantity_(Cincinnati,{j})", 0)
            change_oakland = oakland_quantities[j] - original_variables.get(f"Quantity_(Oakland,{j})", 0)
            print(f"Railroad {j}: Cincinnati Change: {change_cincinnati}, Oakland Change: {change_oakland}")
    print("\n")

Scenario             Objective            Avg Cost per Gallon

Original                  1318984.93            1.71                

| Railroad | Cincinnati Quantity | Oakland Quantity |
|----------|---------------------|------------------|
| 1        | 0.0                 | 22418.0          |
| 2        | 6800.0              | 0.0              |
| 3        | 39636.0             | 40654.0          |
| 4        | 100447.0            | 0.0              |
| 5        | 24570.0             | 0.0              |
| 6        | 0.0                 | 64761.0          |
| 7        | 0.0                 | 33689.0          |
| 8        | 68486.0             | 0.0              |
| 9        | 148586.0            | 0.0              |
| 10       | 111475.0            | 0.0              |
| 11       | 0.0                 | 112000.0         |



Increase Both 10%         1450883.42            1.88                

| Railroad | Cincinnati Quantity | Oakland Quantity |
|----------|---------------------|----

In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value, LpStatus

def solve_model(Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost):
    # Define Environment & Direction of Optimization
    model = LpProblem("Q2", sense=LpMinimize)

    Facility = ['Cincinnati', 'Oakland']
    Railroad = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

    Demand = {1: 22418, 2: 6800, 3: 80290, 4: 100447, 5: 24570, 6: 64761, 7: 33689, 8: 68486, 9: 148586, 10: 111475, 11: 112000}

    # Decision Variables
    Qt = LpVariable.dicts("Quantity", [(i, j) for i in Facility for j in Railroad], lowBound=0, cat='Continuous')

    # Objective Function
    model += lpSum(Qt['Cincinnati', j] * (Prod_Cost['Cincinnati'] + Cincinnati_T_Cost[j]) for j in Railroad) + \
             lpSum(Qt['Oakland', j] * (Prod_Cost['Oakland'] + Oakland_T_Cost[j]) for j in Railroad)

    # Constraints
    for j in Railroad:
        model += lpSum(Qt[i, j] for i in Facility) == Demand[j]

    model += lpSum(Qt['Cincinnati', j] for j in Railroad) <= 500000

    # Solve the Problem
    model.solve()

    return round(value(model.objective), 2), {v.name: round(v.varValue, 2) for v in model.variables()}

# Base Costs
Cincinnati_T_Cost = {1: 10000, 2: 0.84, 3: 0.83, 4: 0.45, 5: 0.36, 6: 10000, 7: 10000, 8: 0.34, 9: 0.34, 10: 0.34, 11: 10000}
Oakland_T_Cost = {1: 0.22, 2: 0.74, 3: 0.49, 4: 10000, 5: 10000, 6: 0.22, 7: 0.22, 8: 10000, 9: 10000, 10: 10000, 11: 0.15}
Prod_Cost = {'Cincinnati': 1.2, 'Oakland': 1.65}

# Increase costs by 10%
Cincinnati_T_Cost_increase_10 = {k: v*1.1 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_increase_10 = {k: v*1.1 for k, v in Oakland_T_Cost.items()}
Prod_Cost_increase_10 = {k: v*1.1 for k, v in Prod_Cost.items()}

# Decrease costs by 10%
Cincinnati_T_Cost_decrease_10 = {k: v*0.9 for k, v in Cincinnati_T_Cost.items()}
Oakland_T_Cost_decrease_10 = {k: v*0.9 for k, v in Oakland_T_Cost.items()}
Prod_Cost_decrease_10 = {k: v*0.9 for k, v in Prod_Cost.items()}

# Total sales (sum of all demand)
total_sale = sum(Demand.values())

# Run models for all 9 scenarios
scenarios = {
    'Original': (Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost),
    'Increase Both 10%': (Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost_increase_10),
    'Decrease Both 10%': (Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost_decrease_10),
    'Increase Tran 10%': (Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost),
    'Decrease Tran 10%': (Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost),
    'Increase Prod 10%': (Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost_increase_10),
    'Decrease Prod 10%': (Cincinnati_T_Cost, Oakland_T_Cost, Prod_Cost_decrease_10),
    'Inc Trans Dec Prod 10%': (Cincinnati_T_Cost_increase_10, Oakland_T_Cost_increase_10, Prod_Cost_decrease_10),
    'Dec Trans Inc Prod 10%': (Cincinnati_T_Cost_decrease_10, Oakland_T_Cost_decrease_10, Prod_Cost_increase_10)
}

# Solve and store results for each scenario
results = {}
for scenario, (cin_cost, oak_cost, prod_cost) in scenarios.items():
    results[scenario] = solve_model(cin_cost, oak_cost, prod_cost)

# Print results with changes compared to the original in a more presentable format
print(f"{'Scenario':<20} {'Objective':<20} {'Avg Cost per Gallon':<15}")
for scenario, (objective, variables) in results.items():
    avg_cost_per_gallon = round(objective / total_sale, 2)
    print(f"\n{scenario:<26} {objective:<21} {avg_cost_per_gallon:<20.2f}")

    # Create columns for quantities of Cincinnati and Oakland
    cincinnati_quantities = {j: 0 for j in range(1, 12)}
    oakland_quantities = {j: 0 for j in range(1, 12)}

    for var, value in variables.items():
        if "Cincinnati" in var:
            index = var.split('_')[-1].replace(')', '')
            cincinnati_quantities[int(index)] = value
        elif "Oakland" in var:
            index = var.split('_')[-1].replace(')', '')
            oakland_quantities[int(index)] = value

    # Print table for each scenario
    print("\n| Railroad | Cincinnati Quantity | Oakland Quantity |")
    print("|----------|---------------------|------------------|")
    for j in range(1, 12):
        print(f"| {j:<8} | {cincinnati_quantities[j]:<19} | {oakland_quantities[j]:<16} |")

Scenario             Objective            Avg Cost per Gallon

Original                   1318984.93            1.71                

| Railroad | Cincinnati Quantity | Oakland Quantity |
|----------|---------------------|------------------|
| 1        | 0.0                 | 22418.0          |
| 2        | 6800.0              | 0.0              |
| 3        | 39636.0             | 40654.0          |
| 4        | 100447.0            | 0.0              |
| 5        | 24570.0             | 0.0              |
| 6        | 0.0                 | 64761.0          |
| 7        | 0.0                 | 33689.0          |
| 8        | 68486.0             | 0.0              |
| 9        | 148586.0            | 0.0              |
| 10       | 111475.0            | 0.0              |
| 11       | 0.0                 | 112000.0         |

Increase Both 10%          1450883.42            1.88                

| Railroad | Cincinnati Quantity | Oakland Quantity |
|----------|---------------------|----

In [11]:
from pulp import *

def solve_model(prod_costs, cinc_t_costs, oak_t_costs):
    model = LpProblem("Q2", sense=LpMinimize)

    Facility = ['Cincinnati', 'Oakland']
    Railroad = {1: 'Santa Ana', 2: 'El Paso', 3: 'Pendleton', 4: 'Houston', 5: 'Kansas City', 6: 'Los Angeles',
                7: 'Glendale', 8: 'Jacksonville', 9: 'Little Rock', 10: 'Bridgeport', 11: 'Sacramento'}
    Demand = {1: 22418, 2: 6800, 3: 80290, 4: 100447, 5: 24570, 6: 64761, 7: 33689, 8: 68486, 9: 148586, 10: 111475, 11: 112000}

    # Decision Variables
    Qt = LpVariable.dicts("Q", [(i, j) for i in Facility for j in Railroad.keys()], lowBound=0, cat='Continuous')

    # Objective Function
    model += lpSum(Qt['Cincinnati', j] * (prod_costs['Cincinnati'] + cinc_t_costs[j]) for j in Railroad.keys()) + \
             lpSum(Qt['Oakland', j] * (prod_costs['Oakland'] + oak_t_costs[j]) for j in Railroad.keys())

    # Constraints
    for j in Railroad.keys():
        model += lpSum(Qt[i, j] for i in Facility) == Demand[j]

    model += lpSum(Qt['Cincinnati', j] for j in Railroad.keys()) <= 500000

    # Solve the Problem
    model.solve()
    return model

def replace_with_railroad_name(var_name, Railroad):
    parts = var_name.split('_')
    if parts[-1].strip('()').isdigit():
        parts[-1] = Railroad[int(parts[-1].strip('()'))] + parts[-1][-1]
    return '_'.join(parts)

def print_results(model, scenario, Railroad):
    # The Status of the Solution
    print(f"\nModel Status for {scenario}: {LpStatus[model.status]}")

    # Prepare lists for Cincinnati and Oakland
    cincinnati_vars = []
    oakland_vars = []
    bridgeport_sacramento_cincinnati = []
    bridgeport_sacramento_oakland = []

    # Separate variables for Cincinnati and Oakland, moving Bridgeport and Sacramento to the bottom
    for v in model.variables():
        var_name_converted = replace_with_railroad_name(v.name, Railroad)
        var_value = v.varValue
        if "Cincinnati" in var_name_converted:
            if "Bridgeport" in var_name_converted or "Sacramento" in var_name_converted:
                bridgeport_sacramento_cincinnati.append((var_name_converted, var_value))
            else:
                cincinnati_vars.append((var_name_converted, var_value))
        elif "Oakland" in var_name_converted:
            if "Bridgeport" in var_name_converted or "Sacramento" in var_name_converted:
                bridgeport_sacramento_oakland.append((var_name_converted, var_value))
            else:
                oakland_vars.append((var_name_converted, var_value))

    # Append Bridgeport and Sacramento at the end
    cincinnati_vars.extend(bridgeport_sacramento_cincinnati)
    oakland_vars.extend(bridgeport_sacramento_oakland)

    # Determine the maximum length for formatting
    max_len = max(len(cincinnati_vars), len(oakland_vars))

    # Pad lists to make them equal length
    while len(cincinnati_vars) < max_len:
        cincinnati_vars.append(("", ""))
    while len(oakland_vars) < max_len:
        oakland_vars.append(("", ""))

    # Print the results in separate columns for Cincinnati and Oakland
    print(f"{'Cincinnati':<35} {'Value':<14} {'Oakland':<31} {'Value'}")
    print(f"{'-'*35} {'-'*14} {'-'*31} {'-'*14}")

    total_cincinnati = 0
    total_oakland = 0

    for (cinc_name, cinc_value), (oak_name, oak_value) in zip(cincinnati_vars, oakland_vars):
        print(f"{cinc_name:<35} {cinc_value:<14} {oak_name:<31} {oak_value}")
        if cinc_value != "":
            total_cincinnati += cinc_value
        if oak_value != "":
            total_oakland += oak_value

    # Print the sums
    print(f"\n{'Total Cincinnati Allocation:':<35} {total_cincinnati:<14} {'Total Oakland Allocation:':<31} {total_oakland:<14}")

    # Display Optimal Value of Objective Function
    print("\nTotal Cost =", value(model.objective))

# Original Costs
Cincinnati_T_Cost = {1: 10000, 2: 0.84, 3: 0.83, 4: 0.45, 5: 0.36, 6: 10000, 7: 10000, 8: 0.34, 9: 0.34, 10: 0.34, 11: 10000}
Oakland_T_Cost = {1: 0.22, 2: 0.74, 3: 0.49, 4: 10000, 5: 10000, 6: 0.22, 7: 0.22, 8: 10000, 9: 10000, 10: 10000, 11: 0.15}
Prod_Cost = {'Cincinnati': 1.2, 'Oakland': 1.65}
Railroad = {1: 'Santa Ana', 2: 'El Paso', 3: 'Pendleton', 4: 'Houston', 5: 'Kansas City', 6: 'Los Angeles',
            7: 'Glendale', 8: 'Jacksonville', 9: 'Little Rock', 10: 'Bridgeport', 11: 'Sacramento'}

# Scenario 1: Increase Cincinnati Costs by 10%
model = solve_model({'Cincinnati': Prod_Cost['Cincinnati'] * 1.1, 'Oakland': Prod_Cost['Oakland']},
                    {k: v * 1.1 for k, v in Cincinnati_T_Cost.items()}, Oakland_T_Cost)
print_results(model, "Increase Cincinnati Costs by 10%", Railroad)

# Scenario 2: Decrease Cincinnati Costs by 10%
model = solve_model({'Cincinnati': Prod_Cost['Cincinnati'] * 0.9, 'Oakland': Prod_Cost['Oakland']},
                    {k: v * 0.9 for k, v in Cincinnati_T_Cost.items()}, Oakland_T_Cost)
print_results(model, "Decrease Cincinnati Costs by 10%", Railroad)

# Scenario 3: Increase Oakland Costs by 10%
model = solve_model({'Cincinnati': Prod_Cost['Cincinnati'], 'Oakland': Prod_Cost['Oakland'] * 1.1},
                    Cincinnati_T_Cost, {k: v * 1.1 for k, v in Oakland_T_Cost.items()})
print_results(model, "Increase Oakland Costs by 10%", Railroad)

# Scenario 4: Decrease Oakland Costs by 10%
model = solve_model({'Cincinnati': Prod_Cost['Cincinnati'], 'Oakland': Prod_Cost['Oakland'] * 0.9},
                    Cincinnati_T_Cost, {k: v * 0.9 for k, v in Oakland_T_Cost.items()})
print_results(model, "Decrease Oakland Costs by 10%", Railroad)


Model Status for Increase Cincinnati Costs by 10%: Optimal
Cincinnati                          Value          Oakland                         Value
----------------------------------- -------------- ------------------------------- --------------
Q_('Cincinnati',_Santa Ana)         0.0            Q_('Oakland',_Santa Ana)        22418.0
Q_('Cincinnati',_El Paso)           6800.0         Q_('Oakland',_El Paso)          0.0
Q_('Cincinnati',_Pendleton)         0.0            Q_('Oakland',_Pendleton)        80290.0
Q_('Cincinnati',_Houston)           100447.0       Q_('Oakland',_Houston)          0.0
Q_('Cincinnati',_Kansas City)       24570.0        Q_('Oakland',_Kansas City)      0.0
Q_('Cincinnati',_Los Angeles)       0.0            Q_('Oakland',_Los Angeles)      64761.0
Q_('Cincinnati',_Glendale)          0.0            Q_('Oakland',_Glendale)         33689.0
Q_('Cincinnati',_Jacksonville)      68486.0        Q_('Oakland',_Jacksonville)     0.0
Q_('Cincinnati',_Little Rock)       14858